In [ ]:
import os
import sys
import json

# Config
os.environ["STEAM__API_KEY"] = "id"
os.environ["FABRIC__WORKSPACE_ID"] = "id"
os.environ["FABRIC__BRONZE_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__SILVER_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__GOLD_LAKEHOUSE_ID"] = "id"

## Run Transformation

In [ ]:
sys.path.insert(0, "/lakehouse/default/Files")

In [ ]:
from src.steam_analytics.config import Settings

settings = Settings()
SILVER_PATH = settings.fabric.silver_abfss_path
GOLD_PATH = settings.fabric.gold_abfss_path

In [ ]:
from src.steam_analytics.transformation.gold import run_gold_transform

results = run_gold_transform(spark, SILVER_PATH, GOLD_PATH)

In [ ]:
from notebookutils import mssparkutils
import json

mssparkutils.notebook.exit(json.dumps({
    "status": "success" if results.all_successful else "partial",
    "total_records": results.total_records,
    "duration_seconds": results.duration_seconds,
}))

## Explore Results

In [ ]:
print("Top 10 Games by Popularity Score:")
df_metrics = spark.read.format("delta").load(f"{GOLD_PATH}/Tables/agg_game_metrics")

df_metrics.select(
    "name", "price_usd", "current_players", "positive_ratio", "popularity_score"
).orderBy("popularity_score", ascending=False).show(10, truncate=False)

In [ ]:
print("Price History (All Changes):")
df_price = spark.read.format("delta").load(f"{GOLD_PATH}/Tables/agg_price_history")

# Show all price points for a specific game
df_price.filter("app_id = 1091500").select(  # Cyberpunk 2077
    "name", "price_final_usd", "discount_tier", "price_effective_from", "is_current"
).orderBy("price_effective_from").show(truncate=False)

In [ ]:
print("Genre Market Analysis:")
df_genre = spark.read.format("delta").load(f"{GOLD_PATH}/Tables/agg_genre_summary")

df_genre.select(
    "genre", "game_count", "avg_price_usd", "avg_positive_ratio", "total_current_players"
).orderBy("game_count", ascending=False).show(15, truncate=False)